# Лабораторна робота № 4

## ІП-14 Бабіч Денис (09.07.2003)

---

# Підготовчий етап

## Імпортування необхідних модулів

In [1]:
import cv2
import numpy as np

## Створення необхідних функцій

In [22]:
def preprocess_frame(frame: np.ndarray, kernel_size: int, threshold1: float, threshold2: float, roi_vertices: list = None) -> np.ndarray:
    preprocessed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if roi_vertices is not None:
        mask = np.zeros_like(preprocessed_frame)
        cv2.fillPoly(mask, np.array([roi_vertices], dtype = np.int32), 255)
        preprocessed_frame = cv2.bitwise_and(preprocessed_frame, mask)

    preprocessed_frame = cv2.GaussianBlur(preprocessed_frame, (kernel_size, kernel_size), 0)
    preprocessed_frame = cv2.Canny(preprocessed_frame, threshold1, threshold2)

    return preprocessed_frame

def determine_obejcts_rectangles(frame: np.ndarray, roi_min: int, roi_max: int, threshold: float = 127) -> list:
    MAXVAL = 255

    objects_rectangles = []

    _, binary_frame = cv2.threshold(frame, threshold, MAXVAL, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if roi_min < area < roi_max:
            x, y, width, height = cv2.boundingRect(contour)
            objects_rectangles.append((x, y, width, height))

    return objects_rectangles

def overlay_objects_rectangles(frame: np.ndarray, rectangles: list, roi_title: str = "", color: tuple = (0, 255, 0), thickness: int = 2) -> np.ndarray:
    ROI_TITLE_OFFSET = 10
    
    for (x, y, width, height) in rectangles:
        cv2.rectangle(frame, (x, y), (x + width, y + height), color, thickness)

        if roi_title:
            cv2.putText(frame, roi_title, (x, y - ROI_TITLE_OFFSET), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame

---

# Основний етап

## Застосування на прикладі відео

In [48]:
KEY_ESCAPE = 27

KERNEL_SIZE = 15
THRESHOLD_1 = 100
THRESHOLD_2 = 100

ROI_MIN = 100
ROI_MAX = 250

ROI_TITLE = "Car"

capture = cv2.VideoCapture("video.mp4")

_, frame = capture.read()

HEIGHT, WIDTH = frame.shape[:2]

THIRD_WIDTH = WIDTH // 3
HALF_HEIGHT = HEIGHT // 2
THIRD_HEIGHT =  HEIGHT // 3

ROI_VERTICES = np.array([
    (0, HEIGHT),
    (THIRD_WIDTH, THIRD_HEIGHT),
    (WIDTH - THIRD_WIDTH, THIRD_HEIGHT),
    (WIDTH, HEIGHT)
], dtype = np.int32)

while capture.isOpened():
    ret, frame = capture.read()

    if not ret:
        break

    preprocessed_frame = preprocess_frame(frame, KERNEL_SIZE, THRESHOLD_1, THRESHOLD_2, ROI_VERTICES)
    roi_rectangles = determine_obejcts_rectangles(preprocessed_frame, ROI_MIN, ROI_MAX)
    processed_frame = overlay_objects_rectangles(frame, roi_rectangles, ROI_TITLE)

    cv2.imshow("", preprocessed_frame)

    if cv2.waitKey(25) & 0xFF == KEY_ESCAPE:
        break

capture.release()
cv2.destroyAllWindows()